In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
import sqlite3

conn = sqlite3.connect('products.db')
cursor = conn.cursor()

In [61]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    description TEXT,
    category_id INTEGER,
    price REAL,
    stock_quantity INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories(id)
)
''')

conn.commit()
conn.close()

In [67]:
import sqlite3

conn = sqlite3.connect('/content/products.db')
cursor = conn.cursor()


In [68]:
import random

In [69]:
# Đọc dữ liệu từ HuggingFace parquet
df = pd.read_parquet("hf://datasets/ShynBui/shopee_products_classification/data/train-00000-of-00001.parquet").sample(200)

df = df.dropna(subset=["id", "name", "description", "label_id", "class_name"])

df['label_id'] = df['label_id'].astype(int)
df['id'] = df['id'].astype(str)

categories_df = df[['label_id', 'class_name']].drop_duplicates().rename(
    columns={'label_id': 'id', 'class_name': 'name'}
)
categories_df['description'] = "Danh mục " + categories_df['name'].str.lower()

products_df = df[['id', 'name', 'description', 'label_id']].copy()
products_df = products_df.rename(columns={'label_id': 'category_id'})
products_df['price'] = products_df['id'].apply(lambda x: round(random.uniform(50_000, 1_500_000), 2))
products_df['stock_quantity'] = products_df['id'].apply(lambda x: random.randint(0, 100))

conn = sqlite3.connect('/content/products.db')
cursor = conn.cursor()

categories_df.to_sql('categories', conn, if_exists='replace', index=False)

products_df.to_sql('products', conn, if_exists='replace', index=False)

print("📂 Danh mục:")
print(pd.read_sql("SELECT * FROM categories LIMIT 5", conn))

print("\n📦 Sản phẩm:")
print(pd.read_sql("SELECT * FROM products LIMIT 5", conn))

categories_df.to_sql('categories', conn, if_exists='append', index=False)
products_df.to_sql('products', conn, if_exists='append', index=False)

conn.commit()
conn.close()

print("Dữ liệu đã được lưu vào SQLite database thành công.")

conn.close()

📂 Danh mục:
   id                         name                           description
0  17                      Sắc Đẹp                      Danh mục sắc đẹp
1  24                       Xe Đạp                       Danh mục xe đạp
2  15  Giặt Giũ & Chăm Sóc Nhà Cửa  Danh mục giặt giũ & chăm sóc nhà cửa
3   7            Balo & Túi Ví Nam            Danh mục balo & túi ví nam
4  12      Máy Ảnh & Máy Quay Phim      Danh mục máy ảnh & máy quay phim

📦 Sản phẩm:
                                               id  \
0              lazadacps_476_4925258281_lazada.vn   
1       cavailles_shopee_322_5836032590_shopee.vn   
2                     tikivn_62_224005697_tiki.vn   
3   waterpush_shopee_pub_322_1754309463_shopee.vn   
4  waterpush_shopee_pub_322_10104801477_shopee.vn   

                                                name  \
0  Máy nấu sáp wax nóng pro 100 tẩy lông chuyên d...   
1   Lốp (Vỏ) xe đạp Deli Tire S601 27.5x1.75 - 1 Cái   
2  Bộ 2 Chai Dung Dịch Tẩy Keo, Nhựa Đường 3M 089.

In [70]:
import sqlite3
import pandas as pd


conn = sqlite3.connect('/content/products.db')


categories_df = pd.read_sql_query("SELECT * FROM categories", conn)
print("📂 Danh mục sản phẩm:")
print(categories_df.head(15))


products_df = pd.read_sql_query("SELECT * FROM products", conn)
print("\n📦 Sản phẩm:")
print(products_df.head(10))

conn.close()


📂 Danh mục sản phẩm:
    id                         name                           description
0   17                      Sắc Đẹp                      Danh mục sắc đẹp
1   24                       Xe Đạp                       Danh mục xe đạp
2   15  Giặt Giũ & Chăm Sóc Nhà Cửa  Danh mục giặt giũ & chăm sóc nhà cửa
3    7            Balo & Túi Ví Nam            Danh mục balo & túi ví nam
4   12      Máy Ảnh & Máy Quay Phim      Danh mục máy ảnh & máy quay phim
5    4                  Giày Dép Nữ                  Danh mục giày dép nữ
6    1               Thời Trang Nam               Danh mục thời trang nam
7    3            Thời Trang Trẻ Em            Danh mục thời trang trẻ em
8   19                      Mẹ & Bé                      Danh mục mẹ & bé
9    2                Thời Trang Nữ                Danh mục thời trang nữ
10  11            Máy Tính & Laptop            Danh mục máy tính & laptop
11  23                       Xe Máy                       Danh mục xe máy
12  20           

In [71]:
len(products_df)

234